In [19]:
from sympy import symbols, Function, diff, Rational, series, Derivative, simplify, factor, Eq, I, exp, collect

# Define the symbols
t = symbols('t')
r = Function('r')(t)
r_dot = Function('r_dot')(t)
theta = Function('theta')(t)
M, c, R, m = symbols('M c R m')

# Define symbols and functions
omega = diff(theta, t)  # Angular velocity
h = symbols('h')  # Angular momentum per unit mass
# Correcting the potential with r instead of R and redoing the computations for the derivatives as in the article
# Now with the updated potential
V = - M / ((1 + (1/2) * (diff(r, t)/c)**2) * r)

# Redo the derivatives of the potential with respect to r and dr/dt as shown in the article using the updated V
partial_V_r = diff(V, r)
d_dt_partial_V_r = diff(partial_V_r, t)

# Now we express the force law corresponding to the new potential, which is the negative gradient of V
force_law = -d_dt_partial_V_r - partial_V_r
# Let's redefine the symbols with 'x' representing (dr/dt)/c
x, y = symbols('x y')

# Replace drdt with x * c in the force law expression
force_law_x = force_law.subs(diff(r, t), x ).subs(diff(x, t), y ).subs(c, 1)
force_law_x = force_law_x.subs(Derivative(x, t), y )
force_law_series = force_law_x.series(x, n=2).removeO()
force_law_series =force_law_series.expand().collect(M/r**2)
force_law_series_simplified = (force_law_series).collect(x)
force_law_series_simplified




# Define the series force law expression (as derived previously)
# For this example, let's use a simplified version without the series expansion details


# Using the complex representation for position z(t)
z = r * exp(I*theta)

# Derivative of z with respect to time, which gives the velocity
z_dot = diff(z, t)

# Derivative of z_dot with respect to time, which gives the acceleration
z_ddot = diff(z_dot, t)

# Extract real and imaginary parts for radial and tangential components
# a_radial = Re(z_ddot), a_tangential = Im(z_ddot)/r
a_radial = diff(r, t, t) - r * omega**2
a_tangential = r * diff(omega, t) + 2 * diff(r, t) * omega

# Now, equate the radial force from the series expansion to the radial acceleration (real part)
radial_force_eq = Eq(a_radial, -force_law_series_simplified)

# There's no tangential force, so we set the tangential acceleration to zero, which leads to angular momentum conservation
angular_momentum_eq = Eq(r**2 * omega, h)

# Define the new variable u = 1/r
u = Function('u')(theta)  # u is a function of theta now, not t
h, omega = symbols('h omega')

# Convert equation 6 to be in terms of u and theta
# We will perform substitutions for r, dr/dt, and d2r/dt2 in terms of u and theta
# dr/dt = -1/u^2 * du/dtheta * dtheta/dt = -1/u^2 * du/dtheta * omega
# d2r/dt2 = derivative of the above expression with respect to t
dr_dt = -h * diff(u, theta)
d2r_dt2 = -h * diff(u, theta, theta) * omega

# Simplify the equations
radial_force_eq_simplified = simplify(radial_force_eq)
angular_momentum_eq_simplified = simplify(angular_momentum_eq).subs(r, 1/u)
radial_force_eq_simplified = radial_force_eq_simplified.subs({x: dr_dt, y: d2r_dt2, r: 1/u})
radial_force_eq_simplified,  angular_momentum_eq_simplified


(Eq(M*(-h*(-1.0*h*omega*Derivative(u(theta(t)), (theta(t), 2))/u(theta(t)) + 2)*Derivative(u(theta(t)), theta(t)) - 1/u(theta(t)))*u(theta(t))**3, -Derivative(1/u(theta(t)), (t, 2)) + Derivative(theta(t), t)**2/u(theta(t))),
 Eq(h, Derivative(theta(t), t)/u(theta(t))**2))

In [20]:
r_prime = -h * diff(u, theta) / (u**2 * omega)
r_double_prime = -h * diff(r_prime, theta) / omega  # chain rule

radial_force_eq_simplified

Eq(M*(-h*(-1.0*h*omega*Derivative(u(theta(t)), (theta(t), 2))/u(theta(t)) + 2)*Derivative(u(theta(t)), theta(t)) - 1/u(theta(t)))*u(theta(t))**3, -Derivative(1/u(theta(t)), (t, 2)) + Derivative(theta(t), t)**2/u(theta(t)))

In [21]:
angular_momentum_eq_simplified

Eq(h, Derivative(theta(t), t)/u(theta(t))**2)

In [22]:
# # Define the symbols and functions needed for the equations
# t = symbols('t')
# r = Function('r')(t)
# theta = Function('theta')(t)
# u = Function('u')(theta)
# M, G, h, omega = symbols('M G h omega')

# Given: radial acceleration equation (from image provided by the user)
# r'' - r * omega**2 = -G * M / r**2 * (1 - 3 * (r')**2 + 6 * r * r')

# Substitute r with 1/u, and r' and r'' with their expressions in terms of u and theta
r_prime = -h * diff(u, theta) / (u**2 * omega)
r_double_prime = -h * diff(r_prime, theta) / omega  # chain rule

# Angular momentum is conserved: h = r^2 * omega
# Substituting expressions in terms of u into the radial acceleration equation
radial_acc_eq = Eq(r_double_prime - (1/u * omega**2), - M * (1 - 3 * r_prime**2 + 6 * (1/u) * r_prime))

# Simplifying and collecting like terms
radial_acc_eq_simplified = simplify(radial_acc_eq.subs({r: 1/u, diff(r, t): r_prime, diff(r, t, t): r_double_prime}))

# The final step would be to express the equation in terms of u and its derivatives with respect to theta
# The derivatives of r with respect to time can be expressed in terms of u and its derivatives with respect to theta
# As per the image instructions, the equation is simplified to a harmonic oscillator form
# A and B are constants, for the general solution of u(theta)

# Express the derivatives in terms of u and theta
r_prime_in_u = -h * diff(u, theta) / omega
r_double_prime_in_u = -h * diff(r_prime_in_u, theta)

# Simplify the equation in terms of u
radial_acc_eq_in_u = simplify(radial_acc_eq_simplified.subs({diff(r, t): r_prime_in_u, diff(r, t, t): r_double_prime_in_u}))

# Now we need to collect and factor terms to identify A and B
# To do this, we will express the equation in the same form as the provided image

# Factor G*M/r**2 out of the equation to identify terms A and B
equation_in_harmonic_oscillator_form = collect(radial_acc_eq_in_u.lhs - radial_acc_eq_in_u.rhs,  M / u**2)

# Now let's find A and B by comparing to the form A*d^2u/dtheta^2 + u = B
# We first isolate the terms with d^2u/dtheta^2 and the term with u alone
coeff_d2u = equation_in_harmonic_oscillator_form.coeff(diff(u, theta, 2))
coeff_u = equation_in_harmonic_oscillator_form.coeff(u)

# Solve for A and B
A = 1 / coeff_d2u
B = -coeff_u / coeff_d2u

A, B


(zoo, nan)

In [24]:
radial_acc_eq

Eq(-h*(-h*Derivative(u(theta(t)), (theta(t), 2))/(omega*u(theta(t))**2) + 2*h*Derivative(u(theta(t)), theta(t))**2/(omega*u(theta(t))**3))/omega - omega**2/u(theta(t)), -M*(-3*h**2*Derivative(u(theta(t)), theta(t))**2/(omega**2*u(theta(t))**4) - 6*h*Derivative(u(theta(t)), theta(t))/(omega*u(theta(t))**3) + 1))

In [25]:
radial_acc_eq_simplified

Eq(3*M*h**2*Derivative(u(theta(t)), theta(t))**2/(omega**2*u(theta(t))**4) + 6*M*h*Derivative(u(theta(t)), theta(t))/(omega*u(theta(t))**3) - M, (h**2*(u(theta(t))*Derivative(u(theta(t)), (theta(t), 2)) - 2*Derivative(u(theta(t)), theta(t))**2) - omega**4*u(theta(t))**2)/(omega**2*u(theta(t))**3))

In [26]:
radial_acc_eq_simplified

Eq(3*M*h**2*Derivative(u(theta(t)), theta(t))**2/(omega**2*u(theta(t))**4) + 6*M*h*Derivative(u(theta(t)), theta(t))/(omega*u(theta(t))**3) - M, (h**2*(u(theta(t))*Derivative(u(theta(t)), (theta(t), 2)) - 2*Derivative(u(theta(t)), theta(t))**2) - omega**4*u(theta(t))**2)/(omega**2*u(theta(t))**3))

In [27]:
radial_acc_eq

Eq(-h*(-h*Derivative(u(theta(t)), (theta(t), 2))/(omega*u(theta(t))**2) + 2*h*Derivative(u(theta(t)), theta(t))**2/(omega*u(theta(t))**3))/omega - omega**2/u(theta(t)), -M*(-3*h**2*Derivative(u(theta(t)), theta(t))**2/(omega**2*u(theta(t))**4) - 6*h*Derivative(u(theta(t)), theta(t))/(omega*u(theta(t))**3) + 1))

In [28]:
radial_acc_eq.subs({r: 1/u, diff(r, t): r_prime, diff(r, t, t): r_double_prime})

Eq(-h*(-h*Derivative(u(theta(t)), (theta(t), 2))/(omega*u(theta(t))**2) + 2*h*Derivative(u(theta(t)), theta(t))**2/(omega*u(theta(t))**3))/omega - omega**2/u(theta(t)), -M*(-3*h**2*Derivative(u(theta(t)), theta(t))**2/(omega**2*u(theta(t))**4) - 6*h*Derivative(u(theta(t)), theta(t))/(omega*u(theta(t))**3) + 1))